In [16]:
import pandas as pd
print(pd.__version__)

import sys,os,copy,pdb,importlib
sys.path.append('../classes')
sys.path.append('../analysis')
import numpy as np
np.set_printoptions(precision=3,suppress=True)
import matplotlib.pyplot as plt
import tournament_new as tn
import create_database as cd
importlib.reload(tn)
import pygame
import beck.beck_game
importlib.reload(beck.beck_game)
from beck.beck_game import BeckGame as Game
from beck.beck_players import HumanBeckPlayer
import arena
importlib.reload(arena)
from arena import Arena

import mcts
from bfts import BFTS

import cog_related
importlib.reload(cog_related)
from cog_related import cog_value_net as cvn


importlib.reload(tn)
game = Game(4,9,4)
all_p = pd.read_pickle(cd.DATABASE_LOC)

from pickle import Pickler, Unpickler
import pickle

# res = tn.merge_res_to_base()
tournament_res = pd.read_pickle('/scratch/zz737/fiar/tournaments/ai_all_player_round_robin_base.pkl')

from utils import *
mctsargs = dotdict({
        'numMCTSSims': 100,
        'cpuct': 2,
        })
import supervised_learning as sl
from importlib import reload
reload(sl)
import keras.backend as K



1.2.0


In [3]:
pip install pygame

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 21.8 MB 31.6 MB/s            
Note: you may need to restart the kernel to use updated packages.


In [27]:
f'{2.1:0.0f}'

'2'

In [10]:
# checkpoint_dir = '/scratch/zz737/fiar/sl/resnet/res15_color_Ex_tournament_6_checkpoints_mcts100_cpuct2_id_1_checkpoint_55'
# file = 'final'
tournament=12
n_res=3
cp = 21

dir_alpha =0
epsilon=0

def get_player(**kwargs):
    tournament = kwargs['tournament']
    n_res = kwargs['n_res']
    cp = kwargs['cp']
    dir_alpha = 0 if 'dir_alpha' not in kwargs.keys() else kwargs['dir_alpha']
    epsilon = 0.25
    cpuct = 2 if 'cpuct' not in kwargs.keys() else kwargs['cpuct']
    mctsargs.cpuct = cpuct
    id = 0 if 'id' not in kwargs.keys() else kwargs['id']
    if tournament=='h':
        human_p = HumanBeckPlayer(game)
        p_human = lambda x:human_p.play(x)
        return None,None,p_human
    else:
#         checkpoint_dir = f'/scratch/zz737/fiar/tournaments/tournament_{tournament}/checkpoints_mcts100_cpuct2_id_res{n_res}-0'
        checkpoint_dir = f'/scratch/zz737/fiar/tournaments/tournament_{tournament:0.0f}/checkpoints_mcts100_cpuct{int(cpuct)}_id_res{n_res:0.0f}-{id:0.0f}'
        if not os.path.exists(checkpoint_dir):
            print(checkpoint_dir)
            checkpoint_dir = f'/scratch/zz737/fiar/tournaments/tournament_{tournament:0.0f}/checkpoints_mcts100_cpuct{cpuct:.0e}_id_res{n_res:0.0f}-{id:0.0f}'

        file =cp if cp=='best' else f'checkpoint_{cp}'

        # checkpoint_dir = '/scratch/zz737/fiar/sl/resnet/res15_color_Ex_tournament_8_checkpoints_mcts100_cpuct2_id_res3-0_checkpoint_61'
        # file = 'final'

        args = pickle.load(open(os.path.join(checkpoint_dir,'args.p'),'rb'))
        on_c = sl.OthelloNNet_resnet(game, args)
        if tournament==8 or tournament==15:
            nnet_c = sl.NNetWrapper(game,nnet=on_c,args=args)
            nnet_c.load_checkpoint(checkpoint_dir, file+'.pth.tar')
            mcts_c = mcts.MCTS(game, nnet_c,mctsargs) 
            ai_c = lambda x:np.random.choice(np.arange(game.getActionSize()),p=mcts_c.getActionProb(x,temp=1/10, dir_alpha=dir_alpha,epsilon=epsilon))

        else:

            nnet_c = sl.NNetWrapper_color(game,nnet=on_c,args=args)
            nnet_c.load_checkpoint(checkpoint_dir, file+'.pth.tar')
            mcts_c = mcts.MCTS_color(game, nnet_c,mctsargs) 
            ai_c = lambda x,c:np.random.choice(np.arange(game.getActionSize()),p=mcts_c.getActionProb(x,c,temp=1/10,dir_alpha=dir_alpha,epsilon=epsilon))

        return nnet_c, mcts_c, ai_c




In [3]:
players = pd.read_pickle('new_t_default_players_updated.p')

In [11]:
players

,tournament,n_res,cp,cpuct,dir_alpha,id
0,8,3,best,2.0,0,0
1,8,9,best,2.0,0,0
2,12,3,55,2.0,0,0
3,12,9,41,2.0,0,0
4,13,3,55,2.0,0,0
5,13,9,39,2.0,0,0
6,14,3,40,2.0,0,0
7,14,9,28,2.0,0,0
8,15,3,35,2.0,0,0
9,15,9,30,2.0,0,0


In [5]:
to_play_inds = [2,1,0,4,8,10]
players.loc[to_play_inds]

,tournament,n_res,cp,cpuct,dir_alpha,id
2,12,3,55,2.0,0,0
1,8,9,best,2.0,0,0
0,8,3,best,2.0,0,0
4,13,3,55,2.0,0,0
8,15,3,35,2.0,0,0
10,16,3,32,2.0,0,0


In [3]:
agents  = [(8,3,'best',2,0,0),(8,9,'best',2,0,0),
           (12,3,58,2,0,0),
           (13,3,55,2,0,0),
           (15,3,37,2,0,0),
           (16,3,32,2,0,0),
           ('h',None,None,None,None,None)
          ]
# cpuct_l = [2]
# dir_alpha_l = [0]

import itertools
# all_agents = list(itertools.product(agents,cpuct_l,dir_alpha_l))
keys = ['tournament','n_res','cp','cpuct','dir_alpha','id']
# all_agents = [[*tup[0],*tup[1:]] for tup in all_agents]
all_agents = agents
players = pd.DataFrame(all_agents,columns=keys)
# players.to_pickle('t8_cpuct_diralpha_tournament_players.p')
t_name = 'play_with_weiji_1-21'
players.to_pickle(f'{t_name}_players.p')

# tour_res = pd.DataFrame(index=players.index,columns=players.index)

# tour_res.to_pickle('t8_cpuct_diralpha_tournament_res.p')
res_name = f'{t_name}_res.p'
# tour_res.to_pickle(res_name)
tour_res = pd.read_pickle(res_name)



In [14]:
players

,tournament,n_res,cp,cpuct,dir_alpha,id
0,8,3,best,2.0,0,0
1,8,9,best,2.0,0,0
2,12,3,55,2.0,0,0
3,12,9,41,2.0,0,0
4,13,3,55,2.0,0,0
5,13,9,39,2.0,0,0
6,14,3,40,2.0,0,0
7,14,9,28,2.0,0,0
8,15,3,35,2.0,0,0
9,15,9,30,2.0,0,0


In [4]:
res_str = {1:'You win', -1:'You lose', 0.0001:'Draw'}

In [12]:
initboard = game.getInitBoard()
initboard[2,1]=1
initboard[1,4]=-1
initboard

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, -1,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0]])

In [15]:
initboard = initboard # None #
id_h = players.loc[players['tournament']=='h']
_,tree_h,p_h = get_player(tournament='h',n_res=None,cp=None)    
track_color_h = False
print('==========Human Playing Frist. Your token is: O  =========')
for id_2,p_dict_2 in players.iloc[3:].iterrows():
    p_dict_2 = dict(p_dict_2)
    if p_dict_2['tournament']=='h':
        pass
    val_2,tree_2,ai_2 = get_player(**p_dict_2)
    track_color_2 = p_dict_2['tournament'] not in (8,15,'h')
#         print(f'{p_dict_1} vs {p_dict_2}')

    arena = Arena(p_h,ai_2, game, display=game.display,tree1=tree_h,tree2=tree_2,track_color=[track_color_h, track_color_2], flip_color=False)
    res=arena.playGame(verbose=True,initboard=initboard)

    tour_res.loc[id_h,id_2] = res
    tour_res.to_pickle(res_name)
    print(f'Result={res_str[res]}. Saved at {res_name}\n')
    K.clear_session()





==========Human Playing Frist. Your token is: O  =========
enter display1


TypeError: display() got an unexpected keyword argument 'gameover'

In [5]:
initboard = None
id_h = players.loc[players['tournament']=='h'].index[0]
_,tree_h,p_h = get_player(tournament='h',n_res=None,cp=None)    
track_color_h = False
print('=========Human Playing Second. Your token is X.  =========')
for id_2,p_dict_2 in players.iterrows():
    p_dict_2 = dict(p_dict_2)
    if p_dict_2['tournament']=='h':
        pass
    val_2,tree_2,ai_2 = get_player(**p_dict_2)
    track_color_2 = p_dict_2['tournament'] not in (8,15,'h')
#         print(f'{p_dict_1} vs {p_dict_2}')

    arena = Arena(ai_2,p_h, game, display=game.display,tree1=tree_2,tree2=tree_h,track_color=[track_color_2, track_color_h], flip_color=False)
    res=arena.playGame(verbose=True,initboard=initboard)

    tour_res.loc[id_2,id_h] = res
    tour_res.to_pickle(res_name)
    print(f'Result={res_str[-res if res!=0.0001 else res]}. Saved at {res_name}\n')
    K.clear_session()






=========Human Playing Second. Your token is X.  =========
Turn  1 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  2 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - O - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
[ 0 0] [ 0 1] [ 0 2] [ 0 3] [ 0 5] [ 0 6] [ 0 7] [ 0 8] [ 1 0] [ 1 1] [ 1 2] [ 1 3] [ 1 4] [ 1 5] [ 1 6] [ 1 7] [ 1 8] [ 2 0] [ 2 1] [ 2 2] [ 2 3] [ 2 4] [ 2 5] [ 2 6] [ 2 7] [ 2 8] [ 3 0] [ 3 1] [ 3 2] [ 3 3] [ 3 4] [ 3 5] [ 3 6] [ 3 7] [ 3 8] 1 4
Turn  3 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - O - - - - |
1 |- - - - X - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  4 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - O - - - - |
1 |- - - - X - - - - |
2 |- - - O - - - - - |
3 |- - - - - - - 

[ 0 0] [ 0 2] [ 0 3] [ 0 6] [ 0 7] [ 0 8] [ 1 0] [ 1 1] [ 1 7] [ 1 8] [ 2 0] [ 2 1] [ 2 2] [ 2 5] [ 2 7] [ 2 8] [ 3 0] [ 3 1] [ 3 2] [ 3 6] [ 3 8] 1 7
Turn  17 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- X - - O X - - - |
1 |- - O X X O O X - |
2 |- - - O X - O - - |
3 |- - - X O O - X - |
-----------------------
Turn  18 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- X - - O X O - - |
1 |- - O X X O O X - |
2 |- - - O X - O - - |
3 |- - - X O O - X - |
-----------------------
[ 0 0] [ 0 2] [ 0 3] [ 0 7] [ 0 8] [ 1 0] [ 1 1] [ 1 8] [ 2 0] [ 2 1] [ 2 2] [ 2 5] [ 2 7] [ 2 8] [ 3 0] [ 3 1] [ 3 2] [ 3 6] [ 3 8] 3 6
Turn  19 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- X - - O X O - - |
1 |- - O X X O O X - |
2 |- - - O X - O - - |
3 |- - - X O O X X - |
-----------------------
Turn  20 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- X - - O X O - - |
1 |- - O X X O O X - |
2 |- - - O X O O - - |
3 |- - - X O O X X - |
----------

Turn  2 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - O - - - - - |
3 |- - - - - - - - - |
-----------------------
[ 0 0] [ 0 1] [ 0 2] [ 0 3] [ 0 4] [ 0 5] [ 0 6] [ 0 7] [ 0 8] [ 1 0] [ 1 1] [ 1 2] [ 1 3] [ 1 4] [ 1 5] [ 1 6] [ 1 7] [ 1 8] [ 2 0] [ 2 1] [ 2 2] [ 2 4] [ 2 5] [ 2 6] [ 2 7] [ 2 8] [ 3 0] [ 3 1] [ 3 2] [ 3 3] [ 3 4] [ 3 5] [ 3 6] [ 3 7] [ 3 8] 1 4
Turn  3 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - X - - - - |
2 |- - - O - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  4 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - X - - - - |
2 |- - - O O - - - - |
3 |- - - - - - - - - |
-----------------------
[ 0 0] [ 0 1] [ 0 2] [ 0 3] [ 0 4] [ 0 5] [ 0 6] [ 0 7] [ 0 8] [ 1 0] [ 1 1] [ 1 2] [ 1 3] [ 1 5] [ 1 6] [ 1 7] [ 1 8] [ 2 0] [ 2 1] [ 2 2] [ 2 5] [ 2 6] [ 2 7] [ 2 8] [ 3 0] [ 3 1] [ 3 2] [ 3 3] [ 3 4] [ 3 5]

[ 2 0] [ 3 0] [ 3 6] 2 0
Turn  35 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |X X X O X O X X X |
1 |O O O X X X O O X |
2 |X X O O O X O O O |
3 |- O X O O X - O X |
-----------------------
Turn  36 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |X X X O X O X X X |
1 |O O O X X X O O X |
2 |X X O O O X O O O |
3 |O O X O O X - O X |
-----------------------
[ 3 6] 3 6
Game over: Turn  36 Result  0.0001
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |X X X O X O X X X |
1 |O O O X X X O O X |
2 |X X O O O X O O O |
3 |O O X O O X X O X |
-----------------------
Result=Draw. Saved at play_with_weiji_1-21_res.p

Turn  1 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  2 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - O - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------

[ 0 0] [ 1 0] [ 1 8] [ 2 0] [ 2 8] [ 3 7] [ 3 8] 3 8
Turn  31 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- O X O O X O X X |
1 |- X O O O X X O - |
2 |- X O X X X O O - |
3 |O X X O O O X - X |
-----------------------
Turn  32 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- O X O O X O X X |
1 |- X O O O X X O - |
2 |- X O X X X O O - |
3 |O X X O O O X O X |
-----------------------
[ 0 0] [ 1 0] [ 1 8] [ 2 0] [ 2 8] 0 0
Turn  33 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |X O X O O X O X X |
1 |- X O O O X X O - |
2 |- X O X X X O O - |
3 |O X X O O O X O X |
-----------------------
Turn  34 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |X O X O O X O X X |
1 |- X O O O X X O - |
2 |- X O X X X O O O |
3 |O X X O O O X O X |
-----------------------
[ 1 0] [ 1 8] [ 2 0] 1 0
Turn  35 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |X O X O O X O X X |
1 |X X O O O X X O - |
2 |- X O X X X O O O |
3 |O X X O O O X O X |


KeyboardInterrupt: Interrupted by user

In [6]:
tour_res

,0,1,2,3,4,5,6
0,NaN,NaN,NaN,NaN,NaN,NaN,1
1,NaN,NaN,NaN,NaN,NaN,NaN,0.0001
2,NaN,NaN,NaN,NaN,NaN,NaN,1
3,NaN,NaN,NaN,NaN,NaN,NaN,-1
4,NaN,NaN,NaN,NaN,NaN,NaN,0.0001
5,NaN,NaN,NaN,NaN,NaN,NaN,0.0001
6,1,NaN,0.0001,NaN,NaN,NaN,NaN


In [8]:
players

,tournament,n_res,cp,cpuct,dir_alpha,id
0,8,3.0,best,2.0,0.0,0.0
1,8,9.0,best,2.0,0.0,0.0
2,12,3.0,58,2.0,0.0,0.0
3,13,3.0,55,2.0,0.0,0.0
4,15,3.0,37,2.0,0.0,0.0
5,16,3.0,32,2.0,0.0,0.0
6,h,NaN,None,NaN,NaN,NaN
